In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout
from tensorflow.keras.layers import add
from tensorflow.keras import optimizers
from nltk.translate.bleu_score import corpus_bleu
import matplotlib.pyplot as plt
import pickle
import time
import glob
import random
import requests
import zipfile
import io
import nltk

In [2]:
def download_and_extract_dataset():
    print("Downloading Flickr8k dataset...")

    # Download Flickr8k images
    image_url = "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip"
    text_url = "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip"

    # Create directories if they don't exist
    if not os.path.exists("data"):
        os.makedirs("data")

    # Download and extract image dataset
    if not os.path.exists("data/Flickr8k_Dataset"):
        print("Downloading image dataset...")
        r = requests.get(image_url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall("data")
        print("Image dataset downloaded and extracted.")
    else:
        print("Image dataset already exists.")

    # Download and extract text dataset
    if not os.path.exists("data/Flickr8k_text"):
        print("Downloading text dataset...")
        r = requests.get(text_url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall("data")
        print("Text dataset downloaded and extracted.")
    else:
        print("Text dataset already exists.")


In [3]:
def load_descriptions(filename):
    file = open(filename, 'r')
    doc = file.read()
    file.close()

    descriptions = {}
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)

        if image_id not in descriptions:
            descriptions[image_id] = []
        descriptions[image_id].append(image_desc)

    return descriptions

In [4]:
def clean_descriptions(descriptions):
    import re
    from nltk.corpus import stopwords

    # Download stopwords
    try:
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('stopwords')

    stop_words = set(stopwords.words('english'))

    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # Convert to lowercase
            desc = desc.lower()
            # Remove punctuation
            desc = re.sub('[^a-zA-Z]', ' ', desc)
            # Remove single characters
            desc = re.sub(r'\s+[a-zA-Z]\s+', ' ', desc)
            # Remove multiple spaces
            desc = re.sub(r'\s+', ' ', desc)
            # Remove stopwords (commented out to keep more natural captions)
            # desc = ' '.join([word for word in desc.split() if word not in stop_words])
            # Store cleaned description
            desc_list[i] = desc

    return descriptions


In [5]:
def save_descriptions(descriptions, filename):
    lines = []
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)

    file = open(filename, 'w')
    file.write(data)
    file.close()

In [6]:
def load_set(filename):
    file = open(filename, 'r')
    doc = file.read()
    file.close()

    dataset = []
    for line in doc.split('\n'):
        if len(line) < 1:
            continue
        image_id = line.split('.')[0]
        dataset.append(image_id)

    return set(dataset)


In [7]:
def load_clean_descriptions(filename, dataset):
    file = open(filename, 'r')
    doc = file.read()
    file.close()

    descriptions = {}
    for line in doc.split('\n'):
        tokens = line.split()
        if len(line) < 2:
            continue
        image_id, image_desc = tokens[0], tokens[1:]
        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = []
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)

    return descriptions

In [8]:
def extract_features(directory, sample_size=None):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    print("VGG16 model loaded for feature extraction")

    features = {}
    image_paths = glob.glob(os.path.join(directory, '*.jpg'))

    if sample_size is not None and sample_size < len(image_paths):
        print(f"Using a sample of {sample_size} images from {len(image_paths)} total images")
        image_paths = random.sample(image_paths, sample_size)

    for i, image_path in enumerate(image_paths):
        if i % 100 == 0:
            print(f"Processing image {i}/{len(image_paths)}")

        image_id = os.path.basename(image_path).split('.')[0]

        try:
            # Load and preprocess the image
            img = load_img(image_path, target_size=(224, 224))
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = tf.keras.applications.vgg16.preprocess_input(img)

            # Extract features
            feature = model.predict(img, verbose=0)

            # Store feature vector
            features[image_id] = feature.flatten()  # Flatten to ensure consistent shape
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            continue

    print(f"Features extracted for {len(features)} images")
    return features


In [9]:
def create_sequences(tokenizer, max_length, descriptions, features, vocab_size):
    X1, X2, y = [], [], []

    for image_id, desc_list in descriptions.items():
        if image_id not in features:
            continue

        feature = features[image_id]

        for desc in desc_list:
            # Tokenize
            seq = tokenizer.texts_to_sequences([desc])[0]

            # Create input-output pairs
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                X1.append(feature)
                X2.append(in_seq)
                y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y)


In [10]:
def define_model(vocab_size, max_length):
    # Feature extractor model
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # Sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    # Disable cuDNN by setting use_cudnn=False
    se3 = LSTM(256, use_cudnn=False)(se2)  # Add this parameter

    # Rest of model definition remains the same
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    print(model.summary())
    return model

In [11]:
def generate_desc(model, tokenizer, photo, max_length):
    """
    Generate a description for an image using the trained model
    """
    # Start the generation process with the start token
    in_text = 'startseq'

    # Iterate until we reach the end token or max length
    for i in range(max_length):
        # Encode the current input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # Pad the sequence
        sequence = pad_sequences([sequence], maxlen=max_length)

        # Reshape photo features if needed
        if isinstance(photo, np.ndarray):
            if len(photo.shape) == 1:  # If 1D array (4096,)
                photo_input = photo.reshape(1, -1)  # Reshape to (1, 4096)
            elif len(photo.shape) == 2 and photo.shape[0] > 1:  # If (4096, 1)
                photo_input = np.transpose(photo)  # Transpose to (1, 4096)
            else:
                photo_input = photo  # Already shaped as (1, 4096)
        else:
            # Handle case where photo might be a tensor
            photo_input = photo

        # Predict the next word
        yhat = model.predict([photo_input, sequence], verbose=0)

        # Convert prediction to word index
        yhat = np.argmax(yhat)

        # Map the index to a word
        word = tokenizer.index_word.get(yhat, '')

        # Stop if word can't be mapped or we reach the end token
        if word == '' or word == 'endseq':
            break

        # Append the word to the current text
        in_text += ' ' + word

    # Remove the start token and return the caption
    final = in_text.replace('startseq', '')

    return final.strip()

In [12]:
def evaluate_model(model, descriptions, features, tokenizer, max_length):
    actual, predicted = [], []

    # Generate captions for all test images
    for image_id, desc_list in descriptions.items():
        if image_id not in features:
            continue

        # Generate description
        yhat = generate_desc(model, tokenizer, features[image_id], max_length)

        # Store actual and predicted
        references = [d.split() for d in desc_list]
        # Clean references by removing start and end tokens
        clean_references = []
        for ref in references:
            ref = [word for word in ref if word not in ('startseq', 'endseq')]
            clean_references.append(ref)

        # Clean hypothesis
        hypothesis = yhat.split()

        # Skip empty hypotheses
        if len(hypothesis) == 0:
            continue

        # Add to list
        actual.append(clean_references)
        predicted.append(hypothesis)

    # Apply smoothing to avoid division by zero
    from nltk.translate.bleu_score import SmoothingFunction
    smooth = SmoothingFunction().method1

    # Calculate BLEU scores with smoothing and error handling
    try:
        bleu1 = corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0),
                           smoothing_function=smooth)
        bleu2 = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0),
                           smoothing_function=smooth)
        bleu3 = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0),
                           smoothing_function=smooth)
        bleu4 = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25),
                           smoothing_function=smooth)
    except Exception as e:
        print(f"Error calculating BLEU scores: {e}")
        # Return zeros on error
        return 0.0, 0.0, 0.0, 0.0

    return bleu1, bleu2, bleu3, bleu4

In [14]:
class RLCaptioningAgent:
    def __init__(self, model, tokenizer, max_length, vocab_size):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.vocab_size = vocab_size
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    def get_reward(self, generated_desc, reference_desc):
        """
        Calculate reward based on BLEU score with error handling
        """
        from nltk.translate.bleu_score import sentence_bleu

        # Clean up descriptions by removing tokens
        generated_words = [w for w in generated_desc.split() if w not in ('startseq', 'endseq')]
        reference_words = [[w for w in desc.split() if w not in ('startseq', 'endseq')]
                          for desc in reference_desc]
        print(f"Generated words: {generated_words}")
        print(f"Reference words: {reference_words}")

        # Check for empty sequences which could cause division by zero
        if len(generated_words) == 0:
            return 0.0  # No words generated, return zero reward

        # Calculate BLEU score with appropriate weights and smoothing
        try:
            from nltk.translate.bleu_score import SmoothingFunction
            smooth = SmoothingFunction().method1  # Apply smoothing to avoid 0/0 fractions
            bleu_score = sentence_bleu(reference_words, generated_words,
                                    weights=(0.25, 0.25, 0.25, 0.25),
                                    smoothing_function=smooth)
            return bleu_score
        except Exception as e:
            print(f"Error calculating BLEU score: {e}")
            return 0.0  # Return zero reward on error

    @tf.function
    def train_step(self, feature, input_seq, target_seq):
        """
        Alternative training step implementation using custom loss calculation
        """
        with tf.GradientTape() as tape:
            # Forward pass
            predictions = self.model([feature, input_seq])  # Shape (1, seq_len, vocab_size)

            # Reshape target to have batch dimension if it doesn't
            if len(target_seq.shape) == 1:
                target_seq = tf.expand_dims(target_seq, 0)  # Add batch dimension

            # One-hot encode target sequences
            target_one_hot = tf.one_hot(target_seq, depth=self.vocab_size)

            # Compute cross-entropy loss manually
            log_probs = tf.math.log(tf.clip_by_value(predictions, 1e-10, 1.0))
            loss = -tf.reduce_sum(target_one_hot * log_probs, axis=-1)

            # Create mask to ignore padding (0)
            mask = tf.cast(tf.not_equal(target_seq, 0), dtype=tf.float32)

            # Apply mask and calculate mean loss
            masked_loss = loss * mask
            loss = tf.reduce_sum(masked_loss) / tf.reduce_sum(mask)

        # Calculate gradients
        gradients = tape.gradient(loss, self.model.trainable_variables)

        # Apply gradients
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

        return loss

    def train(self, features, descriptions, epochs=10, batch_size=32):
        """
        Train the reinforcement learning model with proper tensor handling
        """
        history = []

        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            total_loss = 0.0
            batch_count = 0

            # Shuffle data
            image_ids = list(features.keys())
            random.shuffle(image_ids)

            # Process in batches
            for i in range(0, len(image_ids), batch_size):
                batch_ids = image_ids[i:min(i+batch_size, len(image_ids))]
                batch_loss = 0.0
                valid_samples = 0

                # Process each image in batch
                for image_id in batch_ids:
                    if image_id not in descriptions:
                        continue

                    feature = features[image_id]
                    if isinstance(feature, np.ndarray):
                        # Convert to tensor and ensure shape (1, 4096)
                        if len(feature.shape) == 1:  # (4096,)
                            feature = tf.convert_to_tensor(feature.reshape(1, -1))
                        elif feature.shape[0] > 1 and len(feature.shape) > 1:  # (4096, 1)
                            feature = tf.convert_to_tensor(np.expand_dims(feature, 0))  # Reshape to (1, 4096)
                        else:
                            feature = tf.convert_to_tensor(feature)  # Already (1, 4096)

                    # Process each description for this image
                    for desc in descriptions[image_id]:
                        # Tokenize the description
                        seq = self.tokenizer.texts_to_sequences([desc])[0]

                        # Skip if sequence is too short
                        if len(seq) < 2:
                            continue

                        # Create input and target sequences
                        input_seq = seq[:-1]  # all words except last
                        target_seq = seq[1:]  # all words except first

                        # Pad input sequence
                        input_seq = pad_sequences([input_seq], maxlen=self.max_length)[0]

                        # Convert to tensors
                        input_seq = tf.convert_to_tensor([input_seq])
                        target_seq = tf.convert_to_tensor(target_seq)

                        try:
                            # Perform training step
                            loss = self.train_step(feature, input_seq, target_seq)
                            batch_loss += loss
                            valid_samples += 1
                        except Exception as e:
                            print(f"Error during training: {e}")
                            print(f"Feature shape: {feature.shape if hasattr(feature, 'shape') else 'Unknown'}")
                            print(f"Input sequence shape: {input_seq.shape}")
                            print(f"Target sequence shape: {target_seq.shape}")
                            continue

                # Calculate average batch loss
                if valid_samples > 0:
                    avg_batch_loss = batch_loss / valid_samples
                    total_loss += avg_batch_loss
                    batch_count += 1
                    print(f"  Batch {batch_count}, Loss: {avg_batch_loss:.4f}")

            # Calculate average epoch loss
            if batch_count > 0:
                epoch_loss = total_loss / batch_count
                history.append(epoch_loss)
                print(f"  Epoch {epoch+1}/{epochs}, Average Loss: {epoch_loss:.4f}")

        return history

In [23]:
def main():
    # Create a directory to save models
    if not os.path.exists("models"):
        os.makedirs("models")

    # Download and prepare dataset
    download_and_extract_dataset()

    # Load and clean descriptions
    filename = '/content/data/Flickr8k.token.txt'
    if not os.path.exists(filename):
        print(f"Error: {filename} not found!")
        return

    descriptions = load_descriptions(filename)
    print('Loaded descriptions:', len(descriptions))

    # Clean descriptions
    descriptions = clean_descriptions(descriptions)

    # Save descriptions
    save_descriptions(descriptions, 'descriptions.txt')

    # Load training set (use more images for better results)
    filename = '/content/data/Flickr_8k.trainImages.txt'
    if not os.path.exists(filename):
        print(f"Error: {filename} not found!")
        return

    train = load_set(filename)
    print('Training set size:', len(train))

    # Use more images for training (500 instead of 100)
    train_sample_size = 1000
    train_sample = list(train)[:train_sample_size]
    print(f'Using {len(train_sample)} images for training')

    # Load clean descriptions
    train_descriptions = load_clean_descriptions('descriptions.txt', train_sample)
    print('Descriptions loaded:', len(train_descriptions))

    # Extract features
    image_dir = '/content/data/Flicker8k_Dataset'
    if not os.path.exists(image_dir):
        print(f"Error: Image directory {image_dir} not found!")
        return

    # Extract features for training images
    train_features = extract_features(image_dir, sample_size=train_sample_size)
    print('Features extracted:', len(train_features))

    # Check overlap between descriptions and features
    overlap = set(train_descriptions.keys()) & set(train_features.keys())
    print(f'Overlap between descriptions and features: {len(overlap)} images')

    # Prepare tokenizer
    all_desc = []
    for key in train_descriptions.keys():
        [all_desc.append(d) for d in train_descriptions[key]]

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(all_desc)
    vocab_size = len(tokenizer.word_index) + 1
    print('Vocabulary size:', vocab_size)

    # Find the maximum sequence length
    max_length = max(len(d.split()) for d in all_desc)
    print('Maximum sequence length:', max_length)

    # Save tokenizer
    with open('tokenizer.pkl', 'wb') as f:
        pickle.dump(tokenizer, f)

    # Prepare data for training
    X1, X2, y = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)
    print('Training data shapes:', X1.shape, X2.shape, y.shape)

    # Define the model
    model = define_model(vocab_size, max_length)

    # Train the base model
    print("Training base model...")
    model.fit([X1, X2], y, epochs=20, batch_size=64, verbose=1)

    # Save the base model
    model.save('models/model_base.keras')

    # Load test set for evaluation
    filename = '/content/data/Flickr_8k.testImages.txt'
    if not os.path.exists(filename):
        print(f"Error: {filename} not found!")
        return

    test = load_set(filename)
    print('Test set size:', len(test))

    # Use a subset of test images for evaluation
    test_sample_size = 100
    test_sample = list(test)[:test_sample_size]
    print(f'Using {len(test_sample)} images for testing')

    # Load descriptions for test images
    test_descriptions = load_clean_descriptions('descriptions.txt', test_sample)

    # Extract features for test images
    test_features = extract_features(image_dir, sample_size=test_sample_size)

    # Evaluate base model
    print("Evaluating base model...")
    bleu1, bleu2, bleu3, bleu4 = evaluate_model(
        model, test_descriptions, test_features, tokenizer, max_length)
    print(f'Base Model BLEU Scores: {bleu1:.4f}, {bleu2:.4f}, {bleu3:.4f}, {bleu4:.4f}')

    # Fine-tune with RL
    print("Fine-tuning with reinforcement learning...")
    rl_agent = RLCaptioningAgent(model, tokenizer, max_length, vocab_size)
    history = rl_agent.train(train_features, train_descriptions, epochs=5, batch_size=16)

    # Save the RL fine-tuned model
    rl_agent.model.save('models/model_rl.keras')

    # Evaluate RL model
    print("Evaluating RL model...")
    bleu1, bleu2, bleu3, bleu4 = evaluate_model(
        rl_agent.model, test_descriptions, test_features, tokenizer, max_length)
    print(f'RL Model BLEU Scores: {bleu1:.4f}, {bleu2:.4f}, {bleu3:.4f}, {bleu4:.4f}')

    print("\nSample captions:")
    count = 0

    for image_id in test_features.keys():
        if count == 5:
            break

        # Get reference captions
        refs = [' '.join(ref.split()[1:-1]) for ref in test_descriptions.get(image_id, [])]

        if refs:  # Proceed only if reference captions exist
            # Generate caption using base model
            base_caption = generate_desc(model, tokenizer, test_features[image_id], max_length)
            # Generate caption using RL model
            rl_caption = generate_desc(rl_agent.model, tokenizer, test_features[image_id], max_length)

            print(f"Image {count+1} ({image_id}):")
            print("  References:")
            for ref in refs:
                print(f"    - {ref}")  # Print all reference captions

            print(f"  Base model: {base_caption}")
            print(f"  RL model: {rl_caption}")
            print()

            count += 1  # Increment counter



if __name__ == "__main__":
    main()

Image dataset downloaded and extracted.
Text dataset downloaded and extracted.
Loaded descriptions: 8092
Training set size: 6000
Using 1000 images for training
Descriptions loaded: 1000
VGG16 model loaded for feature extraction
Using a sample of 1000 images from 8091 total images
Processing image 0/1000


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_410']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


Processing image 100/1000
Processing image 200/1000
Processing image 300/1000
Processing image 400/1000
Processing image 500/1000
Processing image 600/1000
Processing image 700/1000
Processing image 800/1000
Processing image 900/1000
Features extracted for 1000 images
Features extracted: 1000
Overlap between descriptions and features: 125 images
Vocabulary size: 3139
Maximum sequence length: 29
Training data shapes: (6795, 4096) (6795, 29) (6795, 3139)


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_32            │ (None, 29)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_31            │ (None, 4096)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_8 (Embedding)   │ (None, 29, 256)        │        803,584 │ input_layer_32[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_16 (Dropout)      │ (None, 4096)           │              0 │ input_layer_31[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_17 (Dropout)      │ (None, 29, 256)        │              0 │ embedding_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_8 (NotEqual)    │ (None, 29)             │              0 │ input_layer_32[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_24 (Dense)          │ (None, 256)            │      1,048,832 │ dropout_16[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_8 (LSTM)             │ (None, 256)            │        525,312 │ dropout_17[0][0],      │
│                           │                        │                │ not_equal_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_8 (Add)               │ (None, 256)            │              0 │ dense_24[0][0],        │
│                           │                        │                │ lstm_8[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_25 (Dense)          │ (None, 256)            │         65,792 │ add_8[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_26 (Dense)          │ (None, 3139)           │        806,723 │ dense_25[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

...

 Trainable params: 3,250,243 (12.40 MB)

 Non-trainable params: 0 (0.00 B)

None
Training base model...
Epoch 1/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 6.1133
Epoch 2/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 4.4986
Epoch 3/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.6658
Epoch 4/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.1919
Epoch 5/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.8014
Epoch 6/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.5396
Epoch 7/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.3067
Epoch 8/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.1100
Epoch 9/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 2.0207
Epoch 10/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.8868
Epoch 11/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.8039
Epoch 12/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.6900
Epoch 13/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.6297
Epoch 14/20
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.5317
Epoch 15/20
10

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_444']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


Features extracted for 100 images
Evaluating base model...
Base Model BLEU Scores: 0.7143, 0.5976, 0.2271, 0.1156
Fine-tuning with reinforcement learning...
Epoch 1/5
  Batch 1, Loss: 11.4439
  Batch 2, Loss: 10.5999
  Batch 3, Loss: 10.1741
  Batch 4, Loss: 9.2907
  Batch 5, Loss: 8.4655
  Batch 6, Loss: 9.1889
  Batch 7, Loss: 7.0164
  Batch 8, Loss: 7.5543
  Batch 9, Loss: 7.1597
  Batch 10, Loss: 8.7772
  Batch 11, Loss: 5.7017
  Batch 12, Loss: 6.8818
  Batch 13, Loss: 6.2479
  Batch 14, Loss: 5.6428
  Batch 15, Loss: 5.1285
  Batch 16, Loss: 5.5956
  Batch 17, Loss: 4.8653
  Batch 18, Loss: 4.8789
  Batch 19, Loss: 5.4718
  Batch 20, Loss: 4.8471
  Batch 21, Loss: 5.4600
  Batch 22, Loss: 5.4996
  Batch 23, Loss: 4.9299
  Batch 24, Loss: 4.9481
  Batch 25, Loss: 5.0075
  Batch 26, Loss: 5.1135
  Batch 27, Loss: 5.3929
  Batch 28, Loss: 4.6762
  Batch 29, Loss: 5.4726
  Batch 30, Loss: 5.9683
  Batch 31, Loss: 5.1100
  Batch 32, Loss: 5.5960
  Batch 33, Loss: 4.6569
  Batch 34, Lo